In [2]:
from transformers import AutoTokenizer
from transformers.models.mpnet.modeling_mpnet import MPNetForSequenceClassification
from transformers import AutoModel , AutoConfig, AutoTokenizer
import torch
import huggingface_hub
import time


2023-01-15 00:41:13.014503: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-15 00:41:13.476713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-15 00:41:13.476736: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-15 00:41:14.590437: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [5]:
class MyModel(torch.nn.Module):
    def __init__(self, NUM_CLASSES,path:str):
        super().__init__()
        self.path=path
        self.config= AutoConfig.from_pretrained(f"{self.path}mpnet-base-snli-mnli/", num_labels=3)
        self.base_model = MPNetForSequenceClassification.from_pretrained(f"{self.path}mpnet-base-snli-mnli/", num_labels=3)
        self.lin_layer = torch.nn.Linear(self.config.hidden_size, NUM_CLASSES)
        self.tokenizer = AutoTokenizer.from_pretrained(f"{self.path}mpnet-base-snli-mnli/",)
    def out(self,prem:str,hyp:str):
        input_pairs = [(prem, hyp)]
        inputs = self.tokenizer(["</s></s>".join(input_pair) for input_pair in input_pairs], return_tensors="pt",padding='max_length')
        logits = self.base_model(**inputs).logits
        probs =  torch.softmax(logits, dim=1).tolist()
        return probs

mod = MyModel(3,"../base/")
print(mod.out(hyp="I am a cat",prem="I am a dog"))

[[0.0011279638856649399, 0.0007713533123023808, 0.9981006979942322]]
